In [38]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

In [39]:
data = pd.read_csv('Daily+temperature+data.csv')

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              730 non-null    object 
 1   Temperature(Min)  730 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.5+ KB


In [41]:
data.head()

,Date,Temperature(Min)
0,01-01-2017,14.3
1,01-02-2017,17.4
2,01-03-2017,18.5
3,01-04-2017,16.8
4,01-05-2017,11.5


In [42]:
data['Date'] = data['Date'].str.replace('/','-')

In [43]:
data.head()

,Date,Temperature(Min)
0,01-01-2017,14.3
1,01-02-2017,17.4
2,01-03-2017,18.5
3,01-04-2017,16.8
4,01-05-2017,11.5


In [44]:
data['Date'] = pd.to_datetime(data['Date'], format = "%m-%d-%Y")

In [45]:
data = data.set_index('Date')

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2017-01-01 to 2018-12-31
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature(Min)  730 non-null    float64
dtypes: float64(1)
memory usage: 11.4 KB


In [47]:
train_win = 511
train = data[:train_win]
test = data[train_win:]

In [48]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
model = SimpleExpSmoothing(train['Temperature(Min)'])
model_fit = model.fit(smoothing_level=0.2, optimized = False)
model_fit.params
y_hat_ses = test.copy()
y_hat_ses['ses_forecast'] = model_fit.forecast(219)

In [49]:
y_hat_ses['ses_forecast']

Date
2018-05-27    9.1997
2018-05-28    9.1997
2018-05-29    9.1997
2018-05-30    9.1997
2018-05-31    9.1997
               ...  
2018-12-27    9.1997
2018-12-28    9.1997
2018-12-29    9.1997
2018-12-30    9.1997
2018-12-31    9.1997
Name: ses_forecast, Length: 219, dtype: float64

In [50]:
from sklearn.metrics import mean_squared_error
rsme = np.sqrt((mean_squared_error(test['Temperature(Min)'],y_hat_ses['ses_forecast'])).round(2))
mape = np.round(np.mean(np.abs(test['Temperature(Min)'] - y_hat_ses['ses_forecast'])/test['Temperature(Min)'])*100,2)
result = pd.DataFrame({'Method':['Simple Moving Average Method'], 'MAPE':[mape],'RSME':[rsme]})
result

,Method,MAPE,RSME
0,Simple Moving Average Method,30.24,3.556684


In [53]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
model = ExponentialSmoothing(np.asarray(test['Temperature(Min)']), seasonal_periods=12, trend = 'additive', seasonal= None)
model_fit = model.fit(smoothing_level=0.5, smoothing_slope = 0.01, optimized = False)
print(model_fit.params)
y_hat_holt = test.copy()
y_hat_holt['ses_forecast'] = model_fit.forecast(219)

{'smoothing_level': 0.5, 'smoothing_slope': 0.01, 'smoothing_seasonal': None, 'damping_slope': nan, 'initial_level': 8.2, 'initial_slope': -2.0999999999999996, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


In [54]:
from sklearn.metrics import mean_squared_error
rsme = np.sqrt((mean_squared_error(test['Temperature(Min)'],y_hat_holt['ses_forecast'])).round(2))
mape = np.round(np.mean(np.abs(test['Temperature(Min)'] - y_hat_holt['ses_forecast'])/test['Temperature(Min)'])*100,2)
result = pd.DataFrame({'Method':['Simple Moving Average Method'], 'MAPE':[mape],'RSME':[rsme]})
result

,Method,MAPE,RSME
0,Simple Moving Average Method,180.04,24.247062
